In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import time
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import RidgeCV
from sklearn.metrics import roc_auc_score
import math as m
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import StandardScaler
import pyreadr


In [2]:
import sklearn as sk


In [8]:
scenarios = pd.read_csv( "syntheticDataScenarios.txt",header =0,sep = " " )

0.018259286880493164


In [4]:
scenarios

,scenario,n,p,s0,SNR,corDesign,rho,pB,s0B
0,1,100,1000,10,0.5,Independence,0.0,0,0
1,2,100,1000,10,1.0,Independence,0.0,0,0
2,3,100,1000,10,1.0,Pairwise,0.5,100,1
3,4,100,1000,10,1.0,Pairwise,0.5,100,2
4,5,100,1000,10,1.0,Pairwise,0.5,100,5
...,...,...,...,...,...,...,...,...,...
1894,1895,300,500,40,4.0,Pairwise,0.7,10,2
1895,1896,300,500,40,4.0,Pairwise,0.7,10,5
1896,1897,300,500,40,4.0,Pairwise,0.9,10,1
1897,1898,300,500,40,4.0,Pairwise,0.9,10,2


In [5]:
scenarios_nosso = scenarios[scenarios["corDesign"]!="Toeplitz"]

In [6]:
scenarios_nosso["corDesign"].value_counts()

Pairwise        1674
Independence     144
Name: corDesign, dtype: int64

In [7]:
beta_value = 3
n_test = 500


# synthetic generation


In [9]:
def data_generation(n, p, s0, SNR, corDesign , pB , rho, s0B, n_test, beta_value ):

    
    n_training = n
    n = n_training + n_test 
    
    if(corDesign=="Independence"):

        X = np.random.randn(n,p)
        gamma = np.zeros(p)
        signals = np.random.choice(range(0,p), size=s0, replace=False)
        gamma[signals] = 1
        beta0 = np.zeros(p)
        beta0[signals] = beta_value
        X = pd.DataFrame(X)
        
    if (corDesign == "Pairwise"):
        
        Block_rho = rho
        n_block = int(p/pB)
        
        def block(n, pB, s0B, Block_rho):           
            signal_idx = np.random.choice(range(0,pB), size=s0B, replace=False)
            Sigma = (np.ones((pB,pB)) -np.eye(pB))*Block_rho + np.eye(pB)
            result =  np.random.multivariate_normal(np.zeros(pB), Sigma, (n))
            result = pd.DataFrame(result)
            
            columns_name = np.zeros(pB)
            columns_name[signal_idx] = 1
            result.columns = columns_name
            return result
            
        X = block(n, pB, s0B, Block_rho)
        if (n_block >= 2):
            for b in range(0,n_block-1):
                
                X = pd.concat([X,block(n, pB, s0B, Block_rho)], axis =1)
    
        beta0 = X.columns.values*beta_value   #########isso n foi passado para função
        
        if(sum(X.columns.values)>s0):
            beta0 = np.zeros(p)
            teste = 0
            for b in range(0,p):
                beta0[b] = X.columns.values[b]*beta_value
                if (X.columns.values[b] == 1):
                    teste = teste +1
                if (teste == s0):
                    break
    
    scaler = StandardScaler()
    
    Xr = X; X_training = Xr.iloc[0:n_training,:]; X_test = Xr.iloc[n_training:n,:]
    X_training = scaler.fit_transform(X_training); X_test=scaler.fit_transform(X_test)
    
    sigmae = float(((beta0@(X_training.T)@X_training@(beta0.T))/(n_training*SNR**2))**(1/2))
    
    Xr= np.vstack([X_training,X_test]) 
    Y0 = Xr@beta0
    Y0_training = Y0[0:n_training]
    e_training = np.random.normal(0, sigmae, n_training)
    Y_training = Y0_training + e_training
    Y_training = Y_training - np.mean(Y_training)
    Y0_test = Y0[n_training:n]
    e_test = np.random.normal(0, sigmae, n_test)
    Y_test = Y0_test + e_test
    Y_test = Y_test - np.mean(Y_test)
    
    if(len(e_training)!=len(Y0_training) or len(e_test) != len(Y0_test)):
        print("Algo deu errado nas dimensões")
        return 1
                
    return ( X_training,X_test,Y_training, Y_test, beta0, sigmae, 50/(p-s0))
        
    

In [4]:
( X_training,X_test,Y_training, Y_test, beta0, sigmae, xlim) = data_generation(n = 20,p= 10, s0 = 4, SNR = 2, corDesign = "Pairwise" , pB = 5  , rho = 0.5, s0B = 2, n_test = 10, beta_value =3 )

## synthetic generation student-error

In [8]:
def data_generation_student(n, p, s0, SNR, corDesign , pB , rho, s0B, n_test, beta_value, df ):
    
    
    n_training = n
    n = n_training + n_test 
    
    if(corDesign=="Independence"):

        X = np.random.randn(n,p)
        gamma = np.zeros(p)
        signals = np.random.choice(range(0,p), size=s0, replace=False)
        gamma[signals] = 1
        beta0 = np.zeros(p)
        beta0[signals] = beta_value
        X = pd.DataFrame(X)
        
    if (corDesign == "Pairwise"):
        
        Block_rho = rho
        n_block = int(p/pB)
        
        def block(n, pB, s0B, Block_rho):           
            signal_idx = np.random.choice(range(0,pB), size=s0B, replace=False)
            Sigma = (np.ones((pB,pB)) -np.eye(pB))*Block_rho + np.eye(pB)
            result =  np.random.multivariate_normal(np.zeros(pB), Sigma, (n))
            result = pd.DataFrame(result)
            
            columns_name = np.zeros(pB)
            columns_name[signal_idx] = 1
            result.columns = columns_name
            return result
            
        X = block(n, pB, s0B, Block_rho)
        if (n_block >= 2):
            for b in range(0,n_block-1):
                
                X = pd.concat([X,block(n, pB, s0B, Block_rho)], axis =1)
    
        beta0 = X.columns.values*beta_value   #########isso n foi passado para função
        
        if(sum(X.columns.values)>s0):
            beta0 = np.zeros(p)
            teste = 0
            for b in range(0,p):
                beta0[b] = X.columns.values[b]*beta_value
                if (X.columns.values[b] == 1):
                    teste = teste +1
                if (teste == s0):
                    break
    
    scaler = StandardScaler()
    
    Xr = X; X_training = Xr.iloc[0:n_training,:]; X_test = Xr.iloc[n_training:n,:]
    X_training = scaler.fit_transform(X_training); X_test=scaler.fit_transform(X_test)
    
    sigmae = float(((beta0@(X_training.T)@X_training@(beta0.T))/(n_training*SNR*2))*(1/2))
    
    Xr= np.vstack([X_training,X_test]) 
    Y0 = Xr@beta0
    Y0_training = Y0[0:n_training]
    
    np.random.standard_t
    e_training = np.random.standard_t( df , n_training)*sigmae
    Y_training = Y0_training + e_training
    Y_training = Y_training - np.mean(Y_training)
    Y0_test = Y0[n_training:n]
    e_test = np.random.standard_t( df, n_test)*sigmae
    Y_test = Y0_test + e_test
    Y_test = Y_test - np.mean(Y_test)
    
    if(len(e_training)!=len(Y0_training) or len(e_test) != len(Y0_test)):
        print("Algo deu errado nas dimensões")
        return 1
                
    return ( X_training,X_test,Y_training, Y_test, beta0, sigmae, 50/(p-s0))

In [76]:
np.savetxt("X.csv", X, delimiter=",")
np.savetxt("X_test.csv", X_test, delimiter=",")



In [79]:

np.savetxt("Y.csv", Y, delimiter=",")
np.savetxt("Y_test.csv", Y_test, delimiter=",")

In [9]:
def Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### founding the alphas necessaries with ten-fold cross validation
    cv = ElasticNetCV(l1_ratio = 1, eps = 0.001, n_alphas=100,cv =10,max_iter=5000 ).fit(X,Y)
######## founding idx of the best alpha
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
        
    solution_path = np.zeros((p,len(alphas)))
    i=0
########## creating a matrix (p x num_alphas) with the beta coefficients of each regression with each alpha
    for temp in alphas:
        regression =  ElasticNet( l1_ratio=1.0, alpha = temp, max_iter= 5000 ).fit(X, Y)
        solution_path[:,i] = regression.coef_ 
        i = i+1
    beta_cv = cv.coef_ ########## best alpha
############## Variable selection ################
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)

    rmse = np.sqrt(((Y_test-X_test@beta_cv-(Y_test.mean()))**2).mean())
    ###rmse = np.sqrt(sum((Y_test - regression.predict(X_test))**2))

##### VARIABLE RANKING #### taking the larger alpha for each predictor, that not results in a null solution for this predictor
    score = np.zeros(p)
    for var_idx in range(0,p):
        
        if((sum((solution_path[var_idx,:]!=0))==0)):  #### if the predictor is null for every alpha
            score[var_idx] = 0
        if(sum((solution_path[var_idx,:]==0))==0):    #### if the predictor is not null for every alpha
            score[var_idx]=alphas[0]
        if(sum( ((solution_path[var_idx,:]!=0))!=0 ) and  (sum((solution_path[var_idx,:]==0))!=0 )): #### else
            idx = sum(solution_path[var_idx,:]==0)  ##### idx of the first alhpa with a nonnull response for this predictor
            if (idx != len(alphas)):
                score[var_idx]=alphas[idx] 
        
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,score, max_fpr = xlim)
    
    #return mcc, ppv, tpr, rmse, pauc, cv.coef_,score
    return mcc, ppv ,tpr , rmse, pauc   
    
    

In [40]:
Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

(0.31980107453341566,
 0.38461538461538464,
 0.5,
 13.976351726795787,
 0.6816046099290781,
 array([ 2.29606596,  0.        ,  0.        , -0.        ,  0.        ,
         0.        , -0.        ,  0.        , -0.        ,  0.        ,
         0.        ,  1.04995887,  0.03466179, -0.        ,  0.        ,
        -1.86841271, -0.        ,  0.        ,  2.1803509 , -0.        ,
         0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        -0.        , -0.        ,  4.66558626,  0.        ,  0.        ,
        -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        -0.        ,  0.        , -0.        ,  0.        ,  1.29609967,
        -0.        , -0.        , -0.        ,  0.        ,  0.83347075,
         0.        ,  0.        , -0.        , -0.        ,  0.        ,
         0.        ,  0.35057315, -0.        ,  0.        ,  0.        ,
         1.11600867, -0.        , -0.        , -0.        ,  0.        ,
         2.51637638,  1.35805678

In [10]:
def Lenet_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### founding the alphas necessaries with ten-fold cross validation
    cv = ElasticNetCV(l1_ratio = 0.6, eps = 0.001, n_alphas=100,cv =10, max_iter=5000).fit(X,Y)
######## founding idx of the best alpha
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
        
    solution_path = np.zeros((p,len(alphas)))
    i=0
########## creating a matrix (p x num_alphas) with the beta coefficients of each regression with each alpha
    for temp in alphas:
        regression =  ElasticNet( l1_ratio=0.6, alpha = temp, max_iter= 5000 ).fit(X, Y)
        solution_path[:,i] = regression.coef_ 
        i = i+1
    beta_cv = cv.coef_ ########## best alpha
############## Variable selection ################
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)

    rmse = np.sqrt(((Y_test-X_test@beta_cv-(Y_test.mean()))**2).mean())
    ###rmse = np.sqrt(sum((Y_test - regression.predict(X_test))**2))

##### VARIABLE RANKING #### taking the larger alpha for each predictor, that not results in a null solution for this predictor
    score = np.zeros(p)
    for var_idx in range(0,p):
        
        if((sum((solution_path[var_idx,:]!=0))==0)):  #### if the predictor is null for every alpha
            score[var_idx] = 0
        if(sum((solution_path[var_idx,:]==0))==0):    #### if the predictor is not null for every alpha
            score[var_idx] = score[var_idx]=alphas[0]
        if(sum( ((solution_path[var_idx,:]!=0))!=0 ) and  (sum((solution_path[var_idx,:]==0))!=0 )): #### else
            idx = sum(solution_path[var_idx,:]==0)  ##### idx of the first alhpa with a nonnull response for this predictor
            if (idx != len(alphas)):
                score[var_idx]=alphas[idx] 
    
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,score, max_fpr = xlim)
    
    return mcc, ppv, tpr, rmse, pauc

            




In [46]:
Lenet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

(0.40405326166118033,
 0.32142857142857145,
 0.9,
 13.699997352792513,
 0.7806365740740742,
 array([ 1.28478491,  0.        ,  0.65600325, -0.        ,  0.46135575,
         0.        , -0.        ,  0.        , -0.        ,  0.        ,
         0.18561871,  1.32148306,  0.64807359, -0.        , -0.        ,
        -1.4635415 , -0.        ,  0.        ,  1.52548425, -0.        ,
         0.        , -0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  2.53121382,  0.02562172,  0.        ,
        -0.        , -0.02165945,  0.        ,  0.        ,  0.1830827 ,
        -0.        ,  0.12342589, -0.        ,  0.        ,  1.21372285,
        -0.        , -0.04685558, -0.        ,  0.08663452,  0.98839877,
         0.        ,  0.13742921, -0.04945872, -0.33795066,  0.49754093,
        -0.        ,  0.68840641, -0.        ,  0.24241178,  0.        ,
         1.08403751, -0.        , -0.        , -0.        ,  0.25791778,
         1.77521384,  1.26459051

In [8]:
def Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### founding the alphas necessaries with ten-fold cross validation
    cv = ElasticNetCV(l1_ratio = 0.3, eps = 0.001, n_alphas=100,cv =10,max_iter= 5000).fit(X,Y)
######## founding idx of the best alpha
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
        
    solution_path = np.zeros((p,len(alphas)))
    i=0
########## creating a matrix (p x num_alphas) with the beta coefficients of each regression with each alpha
    for temp in alphas:
        regression =  ElasticNet( l1_ratio=0.3, alpha = temp, max_iter= 5000).fit(X, Y)
        solution_path[:,i] = regression.coef_ 
        i = i+1
    beta_cv = cv.coef_ ########## best alpha
############## Variable selection ################
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)

#### rmse=sqrt(mean((Y_test-X_test%*%beta.cv-mean(Y_test))^2))  ####
    ###rmse = np.sqrt(sum((Y_test - regression.predict(X_test))**2))
    rmse = np.sqrt(((Y_test-X_test@beta_cv-(Y_test.mean()))**2).mean())
##### VARIABLE RANKING #### taking the larger alpha for each predictor, that not results in a null solution for this predictor
    score = np.zeros(p)
    for var_idx in range(0,p):
        
        if((sum((solution_path[var_idx,:]!=0))==0)):  #### if the predictor is null for every alpha
            score[var_idx] = 0
        if(sum((solution_path[var_idx,:]==0))==0):    #### if the predictor is not null for every alpha
            score[var_idx] = score[var_idx]=alphas[0]
        if(sum( ((solution_path[var_idx,:]!=0))!=0 ) and  (sum((solution_path[var_idx,:]==0))!=0 )): #### else
            idx = sum(solution_path[var_idx,:]==0)  ##### idx of the first alhpa with a nonnull response for this predictor
            if (idx != len(alphas)):
                score[var_idx]=alphas[idx] 
    
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,score, max_fpr = xlim)
    
    return mcc, ppv, tpr, rmse, pauc


In [50]:
Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

(0.07597371763975865,
 0.15873015873015872,
 1.0,
 22.017806241373926,
 0.75525,
 array([ 2.82403395, -0.0223219 ,  3.13033561, -0.97075607,  0.72071655,
         1.06999043,  0.39181205,  0.98518209, -2.18059998,  2.79707396,
         1.72487031,  2.51445484, -0.66647748, -0.72665565,  0.71773789,
        -3.83332348, -2.59737049, -1.31405402,  5.1986422 ,  0.74269654,
         0.96785522, -2.02153557,  0.88740601,  2.16199407, -1.8037942 ,
        -0.82089184, -0.        ,  4.60222285,  1.4687095 ,  0.74969322,
         1.48577669, -0.68578747,  1.30182855,  0.29252549,  1.30417591,
        -0.11418039,  0.19127154,  0.70104043,  0.55345098,  2.80261122,
        -0.62860944, -3.5259549 ,  0.42273746, -0.59602273,  0.75415506,
         0.36250057, -0.21840673, -1.35912477, -1.15710426,  4.29991156,
         1.8720755 ,  0.34674776, -1.06702841,  2.39229875,  1.21308146,
         1.33822524,  0.0366971 , -0.79703222, -0.7126483 ,  0.75564045,
         1.95823357,  4.38337451,  0.784857

In [12]:
def Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### the alphas necessaries with ten-fold cross validation
    lambdas = np.logspace(-2,2.7, num = 150, endpoint = True)
    ######## founding idx of the best alpha
    cv = RidgeCV(alphas = lambdas, cv =10).fit(X,Y)
    beta_cv = cv.coef_

    #### rmse=sqrt(mean((Y_test-X_test%*%beta.cv-mean(Y_test))^2))  ####
    rmse = np.sqrt(((Y_test - cv.predict(X_test))**2).mean())

    ##### VARIABLE RANKING #### using the betas of the best regression model
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1

    pauc = roc_auc_score(beta0,abs(beta_cv), max_fpr = xlim)
    
    return rmse, pauc

In [189]:
results_rid = pd.DataFrame(np.zeros((scenarios_nosso.shape[0],10 )))
for i in range(scenarios_nosso.shape[0]):
    n = scenarios_nosso.iloc[i,1]; p = scenarios_nosso.iloc[i,2]; s0 = scenarios_nosso.iloc[i,3]; SNR = scenarios_nosso.iloc[i,4]
    corDesign = scenarios_nosso.iloc[i,5]; rho =scenarios_nosso.iloc[i,6] ; pB = scenarios_nosso.iloc[i,7]; s0B = scenarios_nosso.iloc[i,8]
    
    (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
    ( rmse, pauc) = Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
    
    results_rid.iloc[i,:] = [n,p,s0, SNR, corDesign, rho, pB, s0B, rmse, pauc ]
    
    print("%i, %i ,%i" %(i,n,p) )

0, 100 ,1000
1, 100 ,1000
2, 100 ,1000
3, 100 ,1000
4, 100 ,1000
5, 100 ,1000
6, 100 ,1000
7, 100 ,1000
8, 100 ,1000
9, 100 ,1000
10, 100 ,1000
11, 100 ,1000
12, 100 ,1000
13, 100 ,1000
14, 100 ,1000
15, 100 ,1000
16, 100 ,1000
17, 100 ,1000
18, 100 ,1000
19, 100 ,1000
20, 100 ,1000
21, 100 ,1000
22, 100 ,1000
23, 100 ,1000
24, 100 ,1000
25, 100 ,1000
26, 100 ,1000
27, 100 ,1000
28, 100 ,1000
29, 100 ,1000
30, 100 ,1000
31, 100 ,1000
32, 100 ,1000
33, 100 ,1000
34, 100 ,1000
35, 100 ,1000
36, 100 ,1000
37, 100 ,1000
38, 100 ,1000
39, 100 ,1000
40, 100 ,1000
41, 100 ,1000
42, 100 ,1000
43, 100 ,1000
44, 100 ,1000
45, 100 ,1000
46, 100 ,1000
47, 100 ,1000
48, 100 ,1000
49, 100 ,1000
50, 100 ,1000
51, 100 ,1000
52, 100 ,1000
53, 100 ,1000
54, 100 ,1000
55, 100 ,1000
56, 100 ,1000
57, 100 ,1000
58, 100 ,1000
59, 100 ,1000
60, 100 ,1000
61, 100 ,1000
62, 100 ,1000
63, 100 ,1000
64, 100 ,1000
65, 100 ,1000
66, 100 ,1000
67, 100 ,1000
68, 100 ,1000
69, 100 ,1000
70, 100 ,1000
71, 100 ,1000
72

KeyboardInterrupt: 

In [52]:
Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

(23.165199193667362,
 0.6315,
 array([ 2.8242577 , -0.41298462,  3.40860277, -1.15624481,  0.63240347,
         1.1238492 ,  0.80178211,  0.99471608, -2.31526537,  2.91974114,
         2.14808249,  2.53346998, -0.9888827 , -1.07262541,  0.78061732,
        -3.92569624, -2.87671303, -1.33783137,  5.44281055,  0.97466425,
         0.9683272 , -2.20910314,  0.9596616 ,  2.33121581, -2.09686238,
        -1.16504928, -0.09727401,  4.40291116,  1.67087058,  0.69421623,
         1.72536561, -0.54577988,  1.40349557,  0.29529054,  1.5386915 ,
        -0.47088648,  0.20723517,  0.89778018,  0.73903779,  2.91542982,
        -0.76566109, -3.65447647,  0.47278566, -0.64349708,  0.62251483,
         0.63985835, -0.43671089, -1.73514239, -1.22156971,  4.55203307,
         2.06108646,  0.28891716, -1.24211612,  2.48878343,  1.04636681,
         1.47563103,  0.30108963, -1.00252695, -0.74474838,  0.84557919,
         1.81616904,  4.72328051,  0.4591426 ,  0.20254776,  1.47769095]))

In [8]:
###################333 função stability ###############
def stability(X,Y, size,steps, nlambdas):
    n, p = X.shape

    selected_variables = np.zeros((p,steps))
    #stability_scores = np.zeros((p, nlambdas))
    
    cv = ElasticNetCV(l1_ratio = 1, eps = 0.001, n_alphas=nlambdas,cv =10, max_iter =5000).fit(X,Y)
    
    alphas = cv.alphas_
    
    ##### tirando os alphas
    
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
    stability_scores = np.zeros((p, len(alphas)))
    
    ################
    
    for idx, alpha, in enumerate(alphas):
    # This is the sampling step, where bootstrap samples are generated
    # and the structure learner is fitted
        for iteration in range(steps):
            bootstrap = np.random.choice(np.arange(n), size= m.ceil(n*size),replace=False)

            X_train = X[bootstrap, :]
            Y_train = Y[bootstrap]

      # Assume scikit-learn implementation
            regression =  ElasticNet( l1_ratio=1.0, alpha = alpha, max_iter = 5000 ).fit(X_train, Y_train)
       
            selected_variables[:, iteration] = (np.abs(regression.coef_) > 1e-4)

    # This is the scoring step, where the final stability
    # scores are computed
        stability_scores[:, idx] = selected_variables.mean(axis=1)

    return stability_scores , alphas

In [9]:
######### pegando os dados 
def stability_func(X,Y,size,steps,nlambdas,beta0, xlim, n, p):
    
    (scores, alphas) = stability(X,Y,size, steps, nlambdas)
    
    beta_full = scores.max(axis=1)
    beta_cv = np.array(list(beta_full))
    beta_cv[beta_cv<0.6]=0
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)
    
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,beta_full, max_fpr = xlim)
    
    return mcc, ppv, tpr, pauc

In [10]:
beta_value = 3
n_test = 500
size = 0.632
steps = 50
nlambdas = 100

# FORMA DO DATA SET

In [40]:
total_results = pd.DataFrame(np.zeros((scenarios_nosso.shape[0],30 )))
for i in range(scenarios_nosso.shape[0]):
    n = scenarios_nosso.iloc[i,1]; p = scenarios_nosso.iloc[i,2]; s0 = scenarios_nosso.iloc[i,3]; SNR = scenarios_nosso.iloc[i,4]
    corDesign = scenarios_nosso.iloc[i,5]; rho =scenarios_nosso.iloc[i,6] ; pB = scenarios_nosso.iloc[i,7]; s0B = scenarios_nosso.iloc[i,8]
    scenario = scenarios_nosso.iloc[i,0]
    
    total_results.iloc[i,0:9] = [scenario, n, p, s0, SNR, corDesign, rho, pB, s0B ]
total_results.columns=["scenario","n","p","s0","SNR","corDesign","rho","pB","s0B","mcc_Lasso","ppv_Lasso","tpr_Lasso","rmse_Lasso", "pauc_Lasso","mcc_Lenet", "ppv_Lenet", "tpr_Lenet", "rmse_Lenet", "pauc_Lenet","mcc_Henet", "ppv_Henet", "tpr_Henet", "rmse_Henet", "pauc_Henet","rmse_Ridge", "pauc_Ridge","mcc_Stab", "ppv_Stab", "tpr_Stab", "pauc_Stab"]
total_results.index = total_results.scenario
    

## forma data set consistencia 

In [14]:
################ Nós usamos para fazer a planilha no começo
total_results = pd.DataFrame(np.zeros((scenarios.shape[0],13)))
for i in range(scenarios.shape[0]):
    n = scenarios.iloc[i,1]; p = scenarios.iloc[i,2]; s0 = scenarios.iloc[i,3]; SNR = scenarios.iloc[i,4]
    corDesign = scenarios.iloc[i,5] ; pB = scenarios.iloc[i,6]; s0B = scenarios.iloc[i,7]
    scenario = scenarios.iloc[i,0]
    
    total_results.iloc[i,0:8] = [scenario, n, p, s0, SNR, corDesign, pB, s0B ]
total_results.columns=["scenario","n","p","s0","SNR","corDesign","pB","s0B","mcc_Lasso","ppv_Lasso","tpr_Lasso","rmse_Lasso", "pauc_Lasso"]
total_results.index = total_results.scenario

# fig 2,5,8

In [10]:
scenarios_fig2 =  scenarios_nosso[scenarios_nosso["corDesign"]=="Independence"] 
scenarios_fig2 = scenarios_fig2[(scenarios_fig2.SNR == 2)|(scenarios_fig2.SNR == 0.5)]
scenarios_fig2 = scenarios_fig2[(scenarios_fig2.n == 300)|(scenarios_fig2.n == 100)]
scenarios_fig2 = scenarios_fig2[(scenarios_fig2.s0 != 20)]
scenarios_fig2.index = range(32)

## Igor aqui, usando pra calcular os de consistencia pra df =9

In [11]:
results_fig2 = pd.DataFrame(np.zeros((scenarios_fig2.shape[0],scenarios_fig2.shape[1]+5)))
results_fig2.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [12]:
for i in range(scenarios_fig2.shape[0]):
    scenario = scenarios_fig2.iloc[i,0] ; n = scenarios_fig2.iloc[i,1]; p = scenarios_fig2.iloc[i,2]; 
    s0 = scenarios_fig2.iloc[i,3]; SNR = scenarios_fig2.iloc[i,4]; corDesign = scenarios_fig2.iloc[i,5]
    rho =scenarios_fig2.iloc[i,6] ; pB = scenarios_fig2.iloc[i,7]; s0B = scenarios_fig2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation_student(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value,9)
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig2.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.006, ppv = 0.010, tpr = 0.019, rmse = 52.561, pauc = 0.510
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.137, ppv = 0.153, tpr = 0.216, rmse = 16.011, pauc = 0.661
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

20,2
20,3
20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.020, ppv = 0.035, tpr = 0.022, rmse = 53.276, pauc = 0.528
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.395, ppv = 0.197, tpr = 0.881, rmse = 14.445, pauc = 0.908
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22


## ridge calculation

In [29]:
results_fig2 = pd.DataFrame(np.zeros((scenarios_fig2.shape[0],scenarios_fig2.shape[1]+2)))
results_fig2.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "rmse", "pauc"]

In [32]:
for i in range(scenarios_fig2.shape[0]):
    scenario = scenarios_fig2.iloc[i,0] ; n = scenarios_fig2.iloc[i,1]; p = scenarios_fig2.iloc[i,2]; 
    s0 = scenarios_fig2.iloc[i,3]; SNR = scenarios_fig2.iloc[i,4]; corDesign = scenarios_fig2.iloc[i,5]
    rho =scenarios_fig2.iloc[i,6] ; pB = scenarios_fig2.iloc[i,7]; s0B = scenarios_fig2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,2)))
    #results_i.columns = [ "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        ( rmse, pauc) = Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    rmse_mean = results_i.iloc[:,0].mean()
    pauc_mean = results_i.iloc[:,1].mean()
    print("rmse = %.3f, pauc = %.3f" %(rmse_mean, pauc_mean))
    
    results_fig2.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B,rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
rmse = 21.599, pauc = 0.580
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
rmse = 10.277, pauc = 0.858
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2,51
2,52
2,53
2,54
2,55
2,56
2,57
2,58
2,59
2,60
2,61
2,62
2,63
rmse = 43.224,

22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23
22,24
22,25
22,26
22,27
22,28
22,29
22,30
22,31
22,32
22,33
22,34
22,35
22,36
22,37
22,38
22,39
22,40
22,41
22,42
22,43
22,44
22,45
22,46
22,47
22,48
22,49
22,50
22,51
22,52
22,53
22,54
22,55
22,56
22,57
22,58
22,59
22,60
22,61
22,62
22,63
rmse = 43.652, pauc = 0.538
23,0
23,1
23,2
23,3
23,4
23,5
23,6
23,7
23,8
23,9
23,10
23,11
23,12
23,13
23,14
23,15
23,16
23,17
23,18
23,19
23,20
23,21
23,22
23,23
23,24
23,25
23,26
23,27
23,28
23,29
23,30
23,31
23,32
23,33
23,34
23,35
23,36
23,37
23,38
23,39
23,40
23,41
23,42
23,43
23,44
23,45
23,46
23,47
23,48
23,49
23,50
23,51
23,52
23,53
23,54
23,55
23,56
23,57
23,58
23,59
23,60
23,61
23,62
23,63
rmse = 20.182, pauc = 0.747
24,0
24,1
24,2
24,3
24,4
24,5
24,6
24,7
24,8
24,9
24,10
24,11
24,12
24,13
24,14
24,15
24,16
24,17
24,18
24,19
24,20
24,21
24,22
24,23
24,24
24,25
24,26
24,27
24,28
24,29
24,30
24,31
24,32
24,33
24

## Henet calculation

In [34]:
results_fig3 = pd.DataFrame(np.zeros((scenarios_fig2.shape[0],scenarios_fig2.shape[1]+5)))
results_fig3.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [36]:
for i in range(scenarios_fig2.shape[0]):
    scenario = scenarios_fig2.iloc[i,0] ; n = scenarios_fig2.iloc[i,1]; p = scenarios_fig2.iloc[i,2]; 
    s0 = scenarios_fig2.iloc[i,3]; SNR = scenarios_fig2.iloc[i,4]; corDesign = scenarios_fig2.iloc[i,5]
    rho =scenarios_fig2.iloc[i,6] ; pB = scenarios_fig2.iloc[i,7]; s0B = scenarios_fig2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv ,tpr , rmse, pauc) = Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig3.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.059, ppv = 0.056, tpr = 0.255, rmse = 21.364, pauc = 0.589
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.169, ppv = 0.040, tpr = 0.983, rmse = 9.606, pauc = 0.864
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2,

20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.169, ppv = 0.092, tpr = 0.719, rmse = 21.242, pauc = 0.716
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.134, ppv = 0.023, tpr = 1.000, rmse = 8.332, pauc = 0.999
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23
22,24

## Lenet calculation

In [1]:
results_fig3 = pd.DataFrame(np.zeros((scenarios_fig2.shape[0],scenarios_fig2.shape[1]+5)))
results_fig3.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

NameError: name 'pd' is not defined

In [ ]:
for i in range(scenarios_fig2.shape[0]):
    scenario = scenarios_fig2.iloc[i,0] ; n = scenarios_fig2.iloc[i,1]; p = scenarios_fig2.iloc[i,2]; 
    s0 = scenarios_fig2.iloc[i,3]; SNR = scenarios_fig2.iloc[i,4]; corDesign = scenarios_fig2.iloc[i,5]
    rho =scenarios_fig2.iloc[i,6] ; pB = scenarios_fig2.iloc[i,7]; s0B = scenarios_fig2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv ,tpr , rmse, pauc) = Lenet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)

        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig3.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.076, ppv = 0.071, tpr = 0.266, rmse = 21.277, pauc = 0.590
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.255, ppv = 0.080, tpr = 0.953, rmse = 8.903, pauc = 0.886
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2,

## Stab calculation

In [ ]:
results_fig2_stab = pd.DataFrame(np.zeros((scenarios_fig2.shape[0],scenarios_fig2.shape[1]+4)))
results_fig2_stab.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr", "pauc"]

In [ ]:
for i in range(scenarios_fig2.shape[0]):
    scenario = scenarios_fig2.iloc[i,0] ; n = scenarios_fig2.iloc[i,1]; p = scenarios_fig2.iloc[i,2]; 
    s0 = scenarios_fig2.iloc[i,3]; SNR = scenarios_fig2.iloc[i,4]; corDesign = scenarios_fig2.iloc[i,5]
    rho =scenarios_fig2.iloc[i,6] ; pB = scenarios_fig2.iloc[i,7]; s0B = scenarios_fig2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,4)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv, tpr, pauc) = stability_func(X,Y,size,steps,nlambdas,beta0, xlim, n, p)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    pauc_mean = results_i.iloc[:,3].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , pauc_mean))
    
    results_fig2_stab.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , pauc_mean ]

## IMPORT RESULTS FROM CSV

In [ ]:
total_results = pd.read_excel('total_results_v11.xlsx', engine='openpyxl', index_col=0, header = 0)

# export data to excel 

In [ ]:
###RIDGE
for i in range(results_fig2.shape[0]):
    scenario = results_fig2.scenario[i]
    total_results.loc[scenario, "rmse_Ridge"] = results_fig2.rmse[i]
    total_results.loc[scenario, "pauc_Ridge"] = results_fig2.pauc[i]

In [ ]:
#####HENET
for i in range(results_fig3.shape[0]):
    scenario = results_fig3.scenario[i]
    total_results.loc[scenario, "mcc_Henet"] = results_fig3.mcc[i]
    total_results.loc[scenario, "ppv_Henet"] = results_fig3.ppv[i]
    total_results.loc[scenario, "tpr_Henet"] = results_fig3.tpr[i]
    total_results.loc[scenario, "rmse_Henet"] = results_fig3.rmse[i]
    total_results.loc[scenario, "pauc_Henet"] = results_fig3.pauc[i]

In [ ]:
#####Stability
for i in range(results_fig2_stab.shape[0]):
    scenario =  results_fig2_stab.scenario[i]
    total_results.loc[scenario, "mcc_Stab"] =  results_fig2_stab.mcc[i]
    total_results.loc[scenario, "ppv_Stab"] =  results_fig2_stab.ppv[i]
    total_results.loc[scenario, "tpr_Stab"] =  results_fig2_stab.tpr[i]
    total_results.loc[scenario, "pauc_Stab"] =  results_fig2_stab.pauc[i]

In [ ]:
total_results.to_excel('total_results_v3.xlsx', index = True)

# fig 4,7,10

In [11]:
scenarios_fig =  scenarios_nosso[scenarios_nosso["corDesign"]=="Pairwise"] 
scenarios_fig = scenarios_fig[scenarios_fig.SNR == 1]
scenarios_fig = scenarios_fig[scenarios_fig.n == 200]
scenarios_fig = scenarios_fig[(scenarios_fig.s0B == 1)|(scenarios_fig.s0B == 5)]
scenarios_fig = scenarios_fig[(scenarios_fig.pB == 10)|(scenarios_fig.pB == 100)]

scenarios_fig_p1 = scenarios_fig[(scenarios_fig.p == 1000)&(scenarios_fig.s0 == 10)]
scenarios_fig_p2 = scenarios_fig[(scenarios_fig.p == 4000)&(scenarios_fig.s0 == 40)]

scenarios_fig_p1.index = range(12)
scenarios_fig_p2.index = range(12)

### stab

In [12]:
beta_value = 3
n_test = 500
size = 0.632
steps = 50
nlambdas = 100

In [13]:
results_fig_p2 = pd.DataFrame(np.zeros((scenarios_fig_p2.shape[0],scenarios_fig_p2.shape[1]+4)))
results_fig_p2.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" ,"pauc"]

In [14]:
for i in range(scenarios_fig_p2.shape[0]):
    scenario = scenarios_fig_p2.iloc[i,0] ; n = scenarios_fig_p2.iloc[i,1]; p = scenarios_fig_p2.iloc[i,2]; 
    s0 = scenarios_fig_p2.iloc[i,3]; SNR = scenarios_fig_p2.iloc[i,4]; corDesign = scenarios_fig_p2.iloc[i,5]
    rho =scenarios_fig_p2.iloc[i,6] ; pB = scenarios_fig_p2.iloc[i,7]; s0B = scenarios_fig_p2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,4)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv, tpr, pauc) = stability_func(X,Y,size,steps,nlambdas,beta0, xlim, n, p)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    pauc_mean = results_i.iloc[:,3].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , pauc_mean))
    
    
    results_fig_p2.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.067, ppv = 0.140, tpr = 0.038, pauc = 0.535
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.096, ppv = 0.179, tpr = 0.058, pauc = 0.549
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2,51
2,52
2,53
2,54
2,55
2,56
2

C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 36.142489623649396, tolerance: 34.98246221465201
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 65.06897222831499, tolerance: 42.563334475311784
  model = cd_fast.enet_coordinate_descent(


5,29
5,30
5,31
5,32
5,33
5,34
5,35
5,36
5,37
5,38
5,39
5,40
5,41
5,42
5,43
5,44
5,45
5,46
5,47
5,48
5,49
5,50
5,51
5,52
5,53
5,54
5,55
5,56
5,57
5,58
5,59
5,60
5,61
5,62
5,63
mcc = 0.054, ppv = 0.112, tpr = 0.032, pauc = 0.532
6,0
6,1
6,2
6,3
6,4
6,5
6,6
6,7
6,8
6,9
6,10
6,11
6,12
6,13
6,14
6,15
6,16
6,17
6,18
6,19
6,20
6,21
6,22
6,23
6,24
6,25
6,26
6,27
6,28
6,29
6,30
6,31
6,32
6,33
6,34
6,35
6,36
6,37
6,38
6,39
6,40
6,41
6,42
6,43
6,44
6,45
6,46
6,47
6,48
6,49
6,50
6,51
6,52
6,53
6,54
6,55
6,56
6,57
6,58
6,59
6,60
6,61
6,62
6,63
mcc = 0.077, ppv = 0.162, tpr = 0.041, pauc = 0.542
7,0
7,1
7,2
7,3
7,4
7,5
7,6
7,7
7,8
7,9
7,10
7,11
7,12
7,13
7,14
7,15
7,16
7,17
7,18
7,19
7,20
7,21
7,22
7,23
7,24
7,25
7,26
7,27
7,28
7,29
7,30
7,31
7,32
7,33
7,34
7,35
7,36
7,37
7,38
7,39
7,40
7,41
7,42
7,43
7,44
7,45
7,46
7,47
7,48
7,49
7,50
7,51
7,52
7,53
7,54
7,55
7,56
7,57
7,58
7,59
7,60
7,61
7,62
7,63
mcc = 0.287, ppv = 0.506, tpr = 0.170, pauc = 0.646
8,0
8,1
8,2
8,3
8,4
8,5
8,6
8,7
8,8
8,9
8,10
8,11

### Lasso

In [32]:
results_fig_p1 = pd.DataFrame(np.zeros((scenarios_fig_p1.shape[0],scenarios_fig_p1.shape[1]+5)))
results_fig_p1.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [33]:
for i in range(scenarios_fig_p1.shape[0]):
    scenario = scenarios_fig_p1.iloc[i,0] ; n = scenarios_fig_p1.iloc[i,1]; p = scenarios_fig_p1.iloc[i,2]; 
    s0 = scenarios_fig_p1.iloc[i,3]; SNR = scenarios_fig_p1.iloc[i,4]; corDesign = scenarios_fig_p1.iloc[i,5]
    rho =scenarios_fig_p1.iloc[i,6] ; pB = scenarios_fig_p1.iloc[i,7]; s0B = scenarios_fig_p1.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig_p1.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.352, ppv = 0.160, tpr = 0.852, rmse = 11.154, pauc = 0.842
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.345, ppv = 0.196, tpr = 0.663, rmse = 18.072, pauc = 0.781
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.731097657938335, tolerance: 12.914702776580562
  model = cd_fast.enet_coordinate_descent(


9,12
9,13
9,14
9,15
9,16
9,17
9,18
9,19
9,20
9,21
9,22
9,23
9,24
9,25
9,26
9,27
9,28
9,29
9,30
9,31
9,32
9,33
9,34
9,35
9,36
9,37
9,38
9,39


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.867908703036989, tolerance: 9.976715973450233
  model = cd_fast.enet_coordinate_descent(


9,40
9,41
9,42
9,43
9,44
9,45
9,46
9,47
9,48
9,49
9,50
9,51
9,52
9,53
9,54
9,55
9,56
9,57
9,58
9,59
9,60
9,61
9,62
9,63
mcc = 0.106, ppv = 0.086, tpr = 0.169, rmse = 21.394, pauc = 0.581
10,0
10,1
10,2
10,3
10,4
10,5
10,6
10,7
10,8
10,9
10,10
10,11
10,12
10,13
10,14
10,15
10,16
10,17
10,18
10,19
10,20
10,21
10,22
10,23
10,24
10,25
10,26
10,27
10,28
10,29
10,30
10,31
10,32
10,33
10,34
10,35
10,36
10,37
10,38
10,39
10,40
10,41
10,42
10,43
10,44
10,45
10,46
10,47
10,48
10,49
10,50
10,51
10,52
10,53
10,54
10,55
10,56
10,57
10,58
10,59
10,60
10,61
10,62
10,63
mcc = 0.260, ppv = 0.131, tpr = 0.586, rmse = 10.877, pauc = 0.748
11,0
11,1
11,2
11,3
11,4
11,5
11,6
11,7
11,8
11,9
11,10
11,11
11,12
11,13
11,14
11,15
11,16
11,17
11,18
11,19
11,20
11,21
11,22
11,23
11,24
11,25
11,26
11,27
11,28
11,29
11,30
11,31
11,32
11,33
11,34
11,35
11,36
11,37
11,38
11,39
11,40
11,41
11,42
11,43
11,44
11,45
11,46
11,47
11,48
11,49
11,50
11,51
11,52
11,53
11,54
11,55
11,56
11,57
11,58
11,59
11,60
11,61
11,62
11,6

...

60,57
60,58
60,59


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15.756637646763238, tolerance: 15.58174369835319
  model = cd_fast.enet_coordinate_descent(


60,60


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22.367419398435004, tolerance: 17.873517645052935
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.685391781501266, tolerance: 17.873517645052935
  model = cd_fast.enet_coordinate_descent(


60,61
60,62
60,63
mcc = 0.123, ppv = 0.111, tpr = 0.206, rmse = 20.983, pauc = 0.579
61,0
61,1
61,2
61,3
61,4
61,5
61,6
61,7
61,8
61,9
61,10
61,11
61,12
61,13
61,14
61,15
61,16
61,17
61,18
61,19
61,20
61,21
61,22
61,23
61,24
61,25
61,26
61,27
61,28
61,29
61,30
61,31
61,32
61,33
61,34
61,35
61,36
61,37
61,38
61,39
61,40
61,41
61,42
61,43
61,44
61,45
61,46
61,47
61,48
61,49
61,50
61,51
61,52
61,53
61,54
61,55
61,56
61,57
61,58
61,59
61,60
61,61
61,62
61,63
mcc = 0.295, ppv = 0.158, tpr = 0.658, rmse = 10.569, pauc = 0.762
62,0
62,1
62,2
62,3
62,4
62,5
62,6
62,7
62,8
62,9
62,10
62,11
62,12
62,13
62,14
62,15
62,16
62,17
62,18
62,19
62,20
62,21
62,22
62,23
62,24
62,25
62,26
62,27
62,28
62,29
62,30
62,31
62,32
62,33
62,34
62,35
62,36
62,37
62,38
62,39
62,40
62,41
62,42
62,43
62,44
62,45
62,46
62,47
62,48
62,49
62,50
62,51
62,52
62,53
62,54
62,55
62,56
62,57
62,58
62,59
62,60
62,61
62,62
62,63
mcc = 0.465, ppv = 0.418, tpr = 0.575, rmse = 21.171, pauc = 0.784


In [34]:
results_fig_p2 = pd.DataFrame(np.zeros((scenarios_fig_p2.shape[0],scenarios_fig_p2.shape[1]+5)))
results_fig_p2.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [35]:
for i in range(scenarios_fig_p2.shape[0]):
    scenario = scenarios_fig_p2.iloc[i,0] ; n = scenarios_fig_p2.iloc[i,1]; p = scenarios_fig_p2.iloc[i,2]; 
    s0 = scenarios_fig_p2.iloc[i,3]; SNR = scenarios_fig_p2.iloc[i,4]; corDesign = scenarios_fig_p2.iloc[i,5]
    rho =scenarios_fig_p2.iloc[i,6] ; pB = scenarios_fig_p2.iloc[i,7]; s0B = scenarios_fig_p2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig_p2.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.136, ppv = 0.139, tpr = 0.241, rmse = 38.011, pauc = 0.551
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.192, ppv = 0.211, tpr = 0.271, rmse = 26.178, pauc = 0.593
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

20,3
20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.336, ppv = 0.152, tpr = 0.802, rmse = 12.034, pauc = 0.849
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.474, ppv = 0.288, tpr = 0.842, rmse = 18.342, pauc = 0.911
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23

C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.603571921463796, tolerance: 8.888591582240302
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.83127178758725, tolerance: 8.888591582240302
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14.130400735007647, tolerance: 8.888591582240302
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarn

38,46
38,47
38,48
38,49
38,50
38,51
38,52
38,53
38,54
38,55
38,56
38,57
38,58
38,59
38,60
38,61
38,62
38,63
mcc = 0.060, ppv = 0.038, tpr = 0.126, rmse = 15.733, pauc = 0.528
39,0
39,1
39,2
39,3
39,4
39,5
39,6
39,7
39,8
39,9
39,10
39,11
39,12
39,13
39,14
39,15
39,16
39,17
39,18
39,19
39,20
39,21
39,22
39,23
39,24
39,25
39,26
39,27
39,28
39,29
39,30
39,31
39,32
39,33
39,34
39,35
39,36
39,37
39,38
39,39
39,40
39,41
39,42
39,43
39,44
39,45
39,46
39,47
39,48
39,49
39,50
39,51
39,52
39,53
39,54
39,55
39,56
39,57
39,58
39,59
39,60
39,61
39,62
39,63
mcc = 0.082, ppv = 0.071, tpr = 0.109, rmse = 30.567, pauc = 0.548
40,0
40,1
40,2
40,3
40,4
40,5
40,6
40,7
40,8
40,9
40,10
40,11
40,12
40,13
40,14
40,15
40,16
40,17
40,18
40,19
40,20
40,21
40,22
40,23
40,24
40,25
40,26
40,27
40,28
40,29
40,30
40,31
40,32
40,33
40,34
40,35
40,36
40,37
40,38
40,39
40,40
40,41
40,42
40,43
40,44
40,45
40,46
40,47
40,48
40,49
40,50
40,51
40,52
40,53
40,54
40,55
40,56
40,57
40,58
40,59
40,60
40,61
40,62
40,63
mcc = 0.15

C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 44.97385064597006, tolerance: 14.526037724502368
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.140025579198344, tolerance: 14.526037724502368
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.17805066547318, tolerance: 14.526037724502368
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWa

50,7
50,8
50,9
50,10
50,11
50,12
50,13
50,14
50,15
50,16
50,17
50,18
50,19
50,20
50,21
50,22
50,23
50,24
50,25
50,26
50,27
50,28
50,29
50,30
50,31
50,32
50,33
50,34
50,35
50,36
50,37
50,38
50,39
50,40
50,41
50,42
50,43
50,44
50,45
50,46
50,47
50,48
50,49
50,50
50,51
50,52
50,53
50,54
50,55
50,56
50,57
50,58
50,59
50,60
50,61
50,62
50,63
mcc = 0.033, ppv = 0.032, tpr = 0.071, rmse = 22.520, pauc = 0.508
51,0
51,1
51,2
51,3
51,4
51,5
51,6
51,7
51,8
51,9
51,10
51,11
51,12
51,13
51,14
51,15
51,16
51,17
51,18
51,19
51,20
51,21
51,22
51,23
51,24
51,25
51,26
51,27
51,28
51,29
51,30
51,31
51,32
51,33
51,34
51,35
51,36
51,37
51,38
51,39
51,40
51,41
51,42
51,43
51,44
51,45
51,46
51,47
51,48
51,49
51,50
51,51
51,52
51,53
51,54
51,55
51,56
51,57
51,58
51,59
51,60
51,61
51,62
51,63
mcc = 0.052, ppv = 0.059, tpr = 0.067, rmse = 44.452, pauc = 0.521
52,0
52,1
52,2
52,3
52,4
52,5
52,6
52,7
52,8
52,9
52,10
52,11
52,12
52,13
52,14
52,15
52,16
52,17
52,18
52,19
52,20
52,21
52,22
52,23
52,24
52,25
52,26
5

#### parte 2 Henet

In [16]:
results_fig_p2 = pd.DataFrame(np.zeros((scenarios_fig_p2.shape[0],scenarios_fig_p2.shape[1]+5)))
results_fig_p2.columns = ["scenario","n","p","s0", "SNR", "corDesign"," rho"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [17]:
for i in range(scenarios_fig_p2.shape[0]):
    scenario = scenarios_fig_p2.iloc[i,0] ; n = scenarios_fig_p2.iloc[i,1]; p = scenarios_fig_p2.iloc[i,2]; 
    s0 = scenarios_fig_p2.iloc[i,3]; SNR = scenarios_fig_p2.iloc[i,4]; corDesign = scenarios_fig_p2.iloc[i,5]
    rho =scenarios_fig_p2.iloc[i,6] ; pB = scenarios_fig_p2.iloc[i,7]; s0B = scenarios_fig_p2.iloc[i,8]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation(n,p, s0, SNR, corDesign , pB , rho , s0B, n_test, beta_value)
    
        (mcc, ppv ,tpr , rmse, pauc) = Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig_p2.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, rho, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.109, ppv = 0.062, tpr = 0.753, rmse = 36.049, pauc = 0.532
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.159, ppv = 0.082, tpr = 0.761, rmse = 25.314, pauc = 0.583
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93.28725754324205, tolerance: 67.94136650348003
  model = cd_fast.enet_coordinate_descent(


6,0
6,1
6,2


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74.81147476413571, tolerance: 66.63040267968195
  model = cd_fast.enet_coordinate_descent(


6,3
6,4


...

50,10
50,11
50,12
50,13
50,14
50,15
50,16
50,17
50,18
50,19
50,20
50,21
50,22
50,23
50,24
50,25
50,26
50,27
50,28
50,29
50,30
50,31
50,32
50,33
50,34
50,35


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 55.942979366528505, tolerance: 15.885416223708075
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.296015128846648, tolerance: 15.885416223708075
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27.89208096991024, tolerance: 15.885416223708075
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceW

50,36
50,37
50,38
50,39
50,40
50,41
50,42
50,43
50,44
50,45
50,46
50,47
50,48
50,49
50,50
50,51
50,52
50,53
50,54
50,55
50,56
50,57
50,58


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 220.55335255181782, tolerance: 15.116528061488454
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.161676016229194, tolerance: 15.116528061488454
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39.27229429792135, tolerance: 15.116528061488454
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceW

50,59
50,60
50,61
50,62
50,63
mcc = 0.043, ppv = 0.018, tpr = 0.401, rmse = 22.018, pauc = 0.504
51,0
51,1
51,2
51,3
51,4
51,5
51,6
51,7
51,8
51,9
51,10
51,11
51,12
51,13
51,14
51,15
51,16
51,17
51,18
51,19
51,20
51,21
51,22
51,23
51,24
51,25
51,26
51,27
51,28
51,29
51,30
51,31
51,32
51,33
51,34
51,35
51,36
51,37
51,38
51,39
51,40
51,41
51,42
51,43
51,44
51,45
51,46
51,47
51,48
51,49
51,50
51,51
51,52
51,53
51,54
51,55
51,56
51,57
51,58
51,59
51,60
51,61
51,62
51,63
mcc = 0.127, ppv = 0.032, tpr = 0.784, rmse = 43.997, pauc = 0.517
52,0
52,1
52,2
52,3
52,4
52,5
52,6
52,7
52,8
52,9
52,10
52,11
52,12
52,13
52,14
52,15
52,16
52,17
52,18
52,19
52,20
52,21
52,22
52,23
52,24
52,25
52,26
52,27
52,28
52,29
52,30
52,31
52,32
52,33
52,34
52,35
52,36
52,37
52,38
52,39
52,40
52,41
52,42
52,43
52,44
52,45
52,46
52,47
52,48
52,49
52,50
52,51
52,52
52,53
52,54
52,55
52,56
52,57
52,58
52,59
52,60
52,61
52,62
52,63
mcc = 0.102, ppv = 0.043, tpr = 0.541, rmse = 25.677, pauc = 0.532
53,0
53,1
53,2
53,3
5

### importing results from excel

In [17]:
total_results = pd.read_excel('total_results_v9.xlsx', engine='openpyxl', index_col=0, header = 0)

#### exporting data to 

In [16]:
########LASSO 
for i in range(results_fig2.shape[0]):
    scenario = results_fig2.scenario[i]
    total_results.loc[scenario, "mcc_Lasso"] = results_fig2.mcc[i]
    total_results.loc[scenario, "ppv_Lasso"] = results_fig2.ppv[i]
    total_results.loc[scenario, "tpr_Lasso"] = results_fig2.tpr[i]
    total_results.loc[scenario, "rmse_Lasso"] = results_fig2.rmse[i]
    total_results.loc[scenario, "pauc_Lasso"] = results_fig2.pauc[i]


In [21]:
######## Henet
for i in range(results_fig_p2.shape[0]):
    scenario = results_fig_p2.scenario[i]
    total_results.loc[scenario, "mcc_Henet"] = results_fig_p2.mcc[i]
    total_results.loc[scenario, "ppv_Henet"] = results_fig_p2.ppv[i]
    total_results.loc[scenario, "tpr_Henet"] = results_fig_p2.tpr[i]
    total_results.loc[scenario, "rmse_Henet"] = results_fig_p2.rmse[i]
    total_results.loc[scenario, "pauc_Henet"] = results_fig_p2.pauc[i]

In [18]:
######## Stability
for i in range(results_fig_p2.shape[0]):
    scenario =  results_fig_p2.scenario[i]
    total_results.loc[scenario, "mcc_Stab"] =  results_fig_p2.mcc[i]
    total_results.loc[scenario, "ppv_Stab"] =  results_fig_p2.ppv[i]
    total_results.loc[scenario, "tpr_Stab"] =  results_fig_p2.tpr[i]
    total_results.loc[scenario, "pauc_Stab"] =  results_fig_p2.pauc[i]


In [17]:
total_results.to_excel('total_results_consistencia_df9.xlsx', index = True)

In [16]:
np.random.choice(range(0,35), size=2, replace=False)

array([32, 14])